# Creating a Client

In [4]:
import os
import json
import requests
from dotenv import load_dotenv
import google.generativeai as genai



In [8]:
text = """beautiful accomodations stayed hotel santa maria novella night wished stayed entirety days florence simply gorgeous accomodations, 
            room stayed cheapest rate available assuming simplest abode offer beautiful hardwood floors bed linens comfortable mattress difficult italy drapes fantastic marble bathroom,
              nicer expecting especially price, location not central 10-minute walk sights close train station away noise bustle busier locations, 
              close upscale shopping lots trattorias, nice oasis hectic sightseeing florence highly recommed hotel hotels higher ranked website,  """

In [6]:
load_dotenv()
genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
 
gemini_model = genai.GenerativeModel('gemini-2.0-flash')

gemini_model.temperature = 0.4


def get_gemini_response(user_prompt):
    response = gemini_model.generate_content(user_prompt)
    return response.text



# Basic prompting

In [3]:
response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api}",
        },
        data=json.dumps({
            "model": "openai/gpt-3.5-turbo",
            "messages": [
            {
                "role": "user",
                "content":  f"Provide the sentiment for {text} as Positive or Negative",
            }
            ]
            
        })
        )

response.json()['choices'][0]['message']['content']

'Positive'

In [11]:
response = get_gemini_response(f"Provide the sentiment for {text} as - 'Positive' or 'Negative'")
response

'Positive\n'

# Defining role for llm

In [ ]:
response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api}",
        },
        data=json.dumps({
            "model": "openai/gpt-3.5-turbo",
            "messages": [
            {
                "role":"system",
                "content":"You are a sentiment analyser who outputs the sentiment of the given review"
            },
            {
                "role": "user",
                "content": f"Provide the Sentiment of {text}",
            }

            ]
            
        })
        )


'The sentiment of the given review is positive. The user appreciates the beautiful accommodations at Hotel Santa Maria Novella, describing them as gorgeous with comfortable amenities such as hardwood floors, comfortable mattress, and a fantastic marble bathroom. They also commend the location for being close to sights and amenities while still providing a peaceful oasis away from the bustle of busier locations. Overall, the user highly recommends the hotel, expressing satisfaction with their stay.'

In [13]:

print(response.json()['choices'][0]['message']['content'])


The sentiment of the given review is positive. The user appreciates the beautiful accommodations at Hotel Santa Maria Novella, describing them as gorgeous with comfortable amenities such as hardwood floors, comfortable mattress, and a fantastic marble bathroom. They also commend the location for being close to sights and amenities while still providing a peaceful oasis away from the bustle of busier locations. Overall, the user highly recommends the hotel, expressing satisfaction with their stay.


# Generating output in JSON

In [4]:


response = requests.post(
        url="https://openrouter.ai/api/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {api}",
        },
        data=json.dumps({
            "model": "openai/gpt-3.5-turbo",
            "messages": [
            {
                "role":"system",
                "content":"You are a sentiment analyser who outputs the sentiment of the review in JSON."
                f"The JSON object must use the schema : {json.dumps({'Sentiment':'Positive/Negative'})}"
            },
            {
                "role": "user",
                "content": f"Provide the Sentiment of {text}",
            }

            ]
            
        })
        )
print(response.json()['choices'][0]['message']['content'])


{
    "Sentiment": "Positive"
}


In [6]:
prompt = f"""Provide the Suggestion to improve areas :, by analysing review: 
        Return the response in JSON format with the structrue:
            *  include the `Areas` as keys and Suggestion as values."""
print(prompt)

Provide the Suggestion to improve areas :, by analysing review: 
        Return the response in JSON format with the structrue:
            *  include the `Areas` as keys and Suggestion as values.


In [ ]:
import json

text ="not impressive stayed park inn end 2006 impressed reviewers, area convenient alexanderplatz railway station resembled building site morning woken early sounds construction, booked expedia room floors 26 35 paid extra 3 nights reservation did not confirm writing assigned room 14th floor refused upgrade grounds hotel unlikely views average, disappointed no tea coffee making facilities room, room rate did not include breakfast did not try hotel breakfast ate locally instead, room small stylish lots mirrors glass chrome marble comfortable wonderful beds cotton bedlinen, reviewers mentioned steam-jet showers n't room, expected rave reviews"

response = get_gemini_response(f"""Provide the Sentiment and all departments/areas which are related to hotel mentioned in given text.
Text = {text}.
Return the response in JSON format with the structrue:
*  include key `Sentiment` with value as 'Positive' or 'Negative'.
*  include key `Areas` with value as a list of departments/areas mentioned in the text.""")
response

'```json\n{\n  "Sentiment": "Negative",\n  "Areas": [\n    "Rooms",\n    "Location",\n    "Breakfast",\n    "Customer Service",\n    "Facilities",\n    "Cleanliness"\n  ]\n}\n```'

In [56]:
data = json.loads(response.split("```")[-2].replace("json", ""))
data

{'Sentiment': 'Negative',
 'Areas': ['Rooms',
  'Location',
  'Breakfast',
  'Customer Service',
  'Facilities',
  'Cleanliness']}

In [60]:
any([1<1 , 2<3])

True

In [ ]:
  
response1 = get_gemini_response(f"Provide the Suggestion to improve areas :{data['Areas']} which are related to hotels, by analysing text: {text}"
                                f"""Return the response in JSON format with the structrue:
*  include the `Areas` as keys and Suggestion as values.""")

print(response1)

```json
{
  "Rooms": "Provide tea and coffee making facilities in all rooms, especially given the room rate. Ensure rooms match descriptions provided by reviewers, including features like steam-jet showers. Consider room upgrades when available, particularly for guests who have paid extra for specific room features. Clarify the floor allocation process during booking and ensure confirmations match the final assignment. Address concerns about room size, especially if advertised differently.",
  "Location": "Acknowledge and mitigate the impact of construction noise on guests. Offer earplugs or alternative room assignments to minimize disturbance. While the location is convenient, address the negative perception of the area resembling a 'building site'.",
  "Breakfast": "Consider including breakfast in more room rates to attract a wider range of guests. Since there's no feedback on the breakfast itself, gather guest feedback on the breakfast quality and options to identify areas for impro

In [72]:
recommendations = ["Gym", "Swimming Pool", "Spa" ]
response1 = get_gemini_response(f"You are a recommendation engine suggest users to visit the given areas of hotel like advertising it."
                                f"Recommend areas: {recommendations}, to user"
                                "Example: {'Swimming pool': 'You must visit our Swimming Pool, it is the best in the city!'}"
                                "Return the response in JSON format with the structrue: "
                                "*  include the ares as keys and recommendation as values."
                                )

print(response1)

```json
{
  "Gym": "Elevate your wellness journey in our state-of-the-art Gym! Equipped with the latest equipment and offering personalized training sessions, it's the perfect place to energize your body and mind.",
  "Swimming Pool": "Dive into relaxation at our magnificent Swimming Pool! Whether you want to swim laps for exercise or simply unwind by the water, our pool offers a refreshing escape for everyone.",
  "Spa": "Indulge in pure bliss at our luxurious Spa! Treat yourself to rejuvenating massages, facials, and body treatments designed to melt away stress and restore your inner radiance. A sanctuary for your senses awaits!"
}
```



In [71]:
print(get_gemini_response(f"You are a resturant manager who gives response for the user who entered the feedback based on the sentiment of the feedback"
f"Give response to {text}"
"Start as 'Dear Guest, Thank you for your feedback...' and also say ' will use them to improve our services and address the issues you raised.'"
"Keep it simple and short, like 3 to 4 sentences."))

Dear Guest,

Thank you for your feedback. We're sorry to hear that your stay at Park Inn wasn't as impressive as you'd hoped, especially concerning the construction noise and room assignment. We appreciate you bringing these points to our attention and will use them to improve our services and address the issues you raised.



# Zero Shot learning

In [ ]:

text = """beautiful accomodations stayed hotel santa maria novella night wished stayed entirety days florence simply gorgeous accomodations, 
            room stayed cheapest rate available assuming simplest abode offer beautiful hardwood floors bed linens comfortable mattress difficult italy drapes fantastic marble bathroom,
              nicer expecting especially price, location not central 10-minute walk sights close train station away noise bustle busier locations, 
              close upscale shopping lots trattorias, nice oasis hectic sightseeing florence highly recommed hotel hotels higher ranked website,  """

def Sentiment_provider(text):
    response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api}",
            },
            data=json.dumps({
                "model": "openai/gpt-3.5-turbo",
                "messages": [
                {
                "role":"system",
                "content":"You are a sentiment analysist who outputs the sentiment and the departments/areas mentioned in review in JSON."
                f"The JSON object must use the schema : {json.dumps({'Sentiment':'Positive/Negative', "Areas":[]})}"
            },
            {
                "role": "user",
                "content": f"Provide the Sentiment of {text}",
            }
                ]
            })
            )
    
    return(response.json()['choices'][0]['message']['content'])

    

In [6]:
sentimen_dict = json.loads(Sentiment_provider(text))
sentimen_dict

{'Sentiment': 'Positive',
 'Areas': ['Accommodations',
  'Hotel Santa Maria Novella',
  'Florence',
  'Room',
  'Hardwood floors',
  'Bed linens',
  'Comfortable mattress',
  'Drapes',
  'Marble bathroom',
  'Price',
  'Location',
  'Train station',
  'Shopping',
  'Trattorias',
  'Sightseeing']}

In [18]:
def Suggestion_provider(Sentiment_dict:dict, text):
    response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api}",
            },
            data=json.dumps({
                "model": "openai/gpt-3.5-turbo",
                "messages": [
                {
                    "role":"system",
                    "content":f"You are a suggestion provider who gives suggestions to improve the areas in {Sentiment_dict['Areas']} using the review {text}, and return it in JSON.\n"
                    f"The JSON object must use the schema : {json.dumps({'Area_name':'Suggestion'})}"
                },
                {
                    "role": "user",
                    "content": f"Provide the Suggestion on how to improve {Sentiment_dict['Areas']}",
                }

                ]
            })
            )
    return json.loads((response.json()['choices'][0]['message']['content']))
s = Suggestion_provider(sentimen_dict, text))

In [ ]:
s = s)

In [24]:
for area in s:
    print(f"\033[1m{area}\033[0m: ")
    print(f"{s[area]}\n")

accommodations: 
Consider adding more variety in room options to cater to different budgets and preferences.

hotel: 
Enhance the overall guest experience by offering additional amenities or services such as a spa or rooftop terrace.

Florence: 
Promote local experiences and tours to help guests immerse themselves in the culture and history of Florence.

hardwood floors: 
Ensure regular maintenance to keep the hardwood floors in pristine condition for a more luxurious feel.

bed linens: 
Invest in high-quality bed linens to elevate the comfort and sleep quality for guests.

comfortable mattress: 
Regularly update mattresses to ensure guests have a restful and rejuvenating sleep experience.

drapes: 
Consider blackout drapes to provide guests with better light control for optimal sleeping conditions.

marble bathroom: 
Add luxury touches such as heated marble floors or rainfall showerheads for a spa-like experience.

price: 
Offer seasonal promotions or packages to provide guests with m

# Few shot learning

In [26]:
def Sentiment_provider(text):
    response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api}",
            },
            data=json.dumps({
                "model": "openai/gpt-3.5-turbo",
                "messages": [
                 {
                "role":"system",
                "content":"You are a sentiment analyser who outputs the sentiment of review"
            },
            {
                "role": "user",
                "content": f"""
                            'The room was good'. the output is : 'Positive',
                            'Food is cold. Price is too much'. The output is: 'Negative',
                            {text}. The output is:
                            """,
            }
                ]
            })
            )
    return (response.json()['choices'][0]['message']['content'])

text = "beautiful accomodations stayed hotel santa maria novella night wished stayed entirety days florence simply gorgeous accomodations"
print(Sentiment_provider(text))

'Positive'


In [27]:
def Sentiment_provider(text):
    response = requests.post(
            url="https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {api}",
            },
            data=json.dumps({
                "model": "openai/gpt-3.5-turbo",
                "messages": [
                 {
                "role":"system",
                "content":"You are a sentiment analyser who outputs the sentiment and the departments/areas mentioned in review in JSON."
                f"The JSON object must use the schema : {json.dumps({'Sentiment':'Positive/Negative', "Areas":[]})}"
            },
            {
                "role": "user",
                "content": f"""
                            'The room was good'. the output is : {json.dumps({'Sentiment':'Positive', 'Areas':['Room']})},
                            'Food is cold. Price is too much'. The output is: {json.dumps({'Sentiment':'Negative', 'Areas':['Resturant', 'Pricing']})},
                            {text}. The output is:
                            """,
            }

                ]
            })
            )
    return (response.json()['choices'][0]['message']['content'])

text = """beautiful accomodations stayed hotel santa maria novella night wished stayed entirety days florence simply gorgeous accomodations
        close upscale shopping lots trattorias, nice oasis hectic sightseeing florence highly recommed hotel hotels higher ranked website"""
print(Sentiment_provider(text))

{"Sentiment": "Positive", "Areas": ["Accommodations", "Hotel", "Santa Maria Novella", "Shopping", "Trattorias", "Sightseeing"]}


In [42]:
import pandas as pd

df = pd.read_csv(r"D:\Projects\SpringBoard_Internship\LLM_Internship\Notebook\Sentiment_data.csv")
df.head()

,User_Id,Cust_name,Cust_email,Feedback,Date,Care_Taker,Emp_id,Age,Contact,Department,Customer Stay Duration (hrs),Customer_N_visit,Cust amount to be paid,Nps,Dining,Room Preference,Sport Activities,Wellness,Pricing
0,User_01,Jared,jared3020@example.com,beautiful accomodations stayed hotel santa mar...,2004-01-03,Jarvis,EMP1029,29,9373413477,Laundry,8.9,8,42.70,-85,Vegetaion,King,Aerobics,Fitness Classes,Weekly discount
1,User_02,Marie,marie3752@example.com,great hotel great location hotel located la ra...,2024-01-21,Malone,EMP2189,67,8659403021,Entertainment,9.2,10,237.41,48,Vegan,Double,Runnin,Spa Treatment,Package deals
2,User_03,Elijah,elijah8826@example.com,beautiful hotel great location like beautifull...,2006-01-27,Barrera,EMP2375,80,9687107606,E-commerce,7.2,2,272.77,93,Vegetaion,Single,Swimming,Body Scrubs,Last-minute deals
3,User_04,Sheryl,sheryl5774@example.com,great deal waikiki trip hawaii outrigger luana...,2023-12-21,Montgomery,EMP1904,90,6609849066,Laundry,8.5,2,483.68,24,Vegan,Twin,Runnin,Meditation Classes,All-inclusive
4,User_05,Jeremy,jeremy8767@example.com,choose airport hotel busy triparound australia...,2006-03-07,Houston,EMP1448,39,655389336,E-commerce,6.7,4,338.33,-5,Vegan,Twin,Aerobics,Sauna,Member-exclusive rates


In [43]:
data_batch = {i: df.Feedback[j:j+30].values for i,j in enumerate(range(0,len(df), 30))}
data_batch[0]

array(['beautiful accomodations stayed hotel santa maria novella night wished stayed entirety days florence simply gorgeous accomodations, room stayed cheapest rate available assuming simplest abode offer beautiful hardwood floors bed linens comfortable mattress difficult italy drapes fantastic marble bathroom, nicer expecting especially price, location not central 10-minute walk sights close train station away noise bustle busier locations, close upscale shopping lots trattorias, nice oasis hectic sightseeing florence highly recommed hotel hotels higher ranked website,  ',
       'great hotel great location hotel located la rambla city centre close access shops tour buses.the room immaculate 42 plasma screen screen bathroom.there nice blue ambient lighting room good amenities.the bathroom large equipped.concierge staff overall friendly helpful recommended lot restaurants night spots visit absolutely wonderful.there business centre use free open 24 hours.the thing disliked face in-room

In [49]:
sents = []
system_messege = f"You are the sentiment analyser who takes the list containing review texts and provode corresponding sentiment output JSON,JSON schemaa is {json.dumps({'text starting word':'Sentiment'})}"
chat_completion = client.chat.completions.create(
        messages=[
            {
                "role":"system",
                "content":system_messege
            },
            {
                "role": "user",
                "content": f"Provide the Sentiment of {data_batch[0]}",
            }
        ],
        model="llama3-8b-8192",
        response_format={"type": "json_object"}
        
    )
x = chat_completion.choices[0].message.content




In [6]:
def sentiment(text):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role":"system",
                "content":"You are a sentiment analyser who outputs the sentiment of the review in JSON."
                f"The JSON object must use the schema : {json.dumps({'Sentiment':'Positive/Negative'})}"
            },
            {
                "role": "user",
                "content": f"Provide the Sentiment of {text}",
            }
        ],
        model="llama3-8b-8192",
        response_format={"type": "json_object"}
    )
    out = json.loads(chat_completion.choices[0].message.content)
    return out['Sentiment']

In [44]:
sample_df = df.sample(150)
sample_df

,User_Id,Cust_name,Cust_email,Feedback,Date,Care_Taker,Emp_id,Age,Contact,Department,Customer Stay Duration (hrs),Customer_N_visit,Cust amount to be paid,Nps,Dining,Room Preference,Sport Activities,Wellness,Pricing
5128,User_05129,Ashlee,ashlee7565@example.com,not impressive stayed park inn end 2006 impres...,2021-02-01,Carey,EMP2459,47,7151171276,Accounting,5.3,9,380.82,-33,Vegan,Twin,Soccer,Body Scrubs,Mid-range
9798,User_09799,Nichole,nichole7683@example.com,"wonderful hotel, sister stayed affinia enjoyed...",2007-05-06,Patel,EMP2356,56,8450434956,Banking,6.5,3,378.08,-4,Vegetaion,King,Runnin,Meditation Classes,Mid-range
788,User_0789,Kimberly,kimberly1142@example.com,"derag hotel berlin booked hotel derag website,...",2022-10-13,Murillo,EMP1381,36,7308187463,E-commerce,9.6,2,140.76,-52,Vegetaion,Suite,Runnin,Spa Treatment,Mid-range
6939,User_06940,Julian,julian2606@example.com,"fabulous, booking vacation 4 dismay realized p...",2025-01-28,Sims,EMP1512,29,7864757032,Maintenance,3.6,6,267.42,-60,Non-Vegetarion,Double,Volleyball,Herbal Baths,Seasonal pricing
7130,User_07131,Evan,evan7559@example.com,"great time, stayed hotel late fantastic time, ...",2013-08-06,Li,EMP2779,21,6847401587,Photography,4.2,5,235.04,-68,Vegetaion,Double,Aerobics,Steam Room,Kids stay free
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5363,User_05364,Drew,drew1187@example.com,"great, booked hotel hotelconnect.co.uk best pr...",2024-11-14,Ball,EMP2116,61,6437445093,Entertainment,4.7,7,314.10,36,Vegan,Family,Golf,Personal Training,Package deals
8097,User_08098,Vicki,vicki2203@example.com,great location shame staff write sit room 9am ...,2011-03-24,Parks,EMP1516,53,6299245056,Banking,9.3,7,255.18,-71,Vegan,Double,Golf,Body Scrubs,Kids stay free
8394,User_08395,Ariel,ariel7817@example.com,excellent choice sofitel certainly royal venue...,2002-01-22,Wright,EMP2251,32,6424393219,HealthcareHelp Desk,6.9,9,463.94,-1,Vegan,King,Basketball,Steam Room,Member-exclusive rates
2600,User_02601,Tommy,tommy8422@example.com,fun weekend adams mark n't know definitely sta...,2007-11-10,Morse,EMP1022,77,9350438633,Advertising,1.3,6,196.10,50,Vegetaion,Double,Tennis,Facials,Member-exclusive rates


In [12]:
size = 50
batch_df = [sample_df.Feedback[i:i+size] for i in range(0,150, size)]
sentiment_ls = []
c = 0
for batch in batch_df:
    sent = [sentiment(i) for i in batch]
    sentiment_ls.extend(sent)
    print(c)
    c+=1

0
1
2


In [ ]:
sentiment_ls = []
i = 0
for text in sample_df.Feedback:
    # Using the get_gemini_response function to get sentiment analysis
    # This is an alternative to using the OpenRouter API
    response = get_gemini_response(f"""Analyse the Sentiment for given text.
    Text = {text}.
    Return the response in JSON format with the structrue:
    *  include key `Sentiment` with any of the following values: 'Positive', 'Negative'.""")


    sent = response.split("```")[-2].replace("json", "")
    sentiment_ls.append([text, json.loads(sent)['Sentiment']])
    
sentiment_ls

KeyboardInterrupt: 

In [49]:
sentiment_ls

[["not impressive stayed park inn end 2006 impressed reviewers, area convenient alexanderplatz railway station resembled building site morning woken early sounds construction, booked expedia room floors 26 35 paid extra 3 nights reservation did not confirm writing assigned room 14th floor refused upgrade grounds hotel unlikely views average, disappointed no tea coffee making facilities room, room rate did not include breakfast did not try hotel breakfast ate locally instead, room small stylish lots mirrors glass chrome marble comfortable wonderful beds cotton bedlinen, reviewers mentioned steam-jet showers n't room, expected rave reviews,  ",
  'Negative'],
 ['wonderful hotel, sister stayed affinia enjoyed stay, stay anytime new york, rooms nice clean, staff friendly hotel close walk nearby attractions, great time,  ',
  'Positive'],
 ["derag hotel berlin booked hotel derag website, running summer special 55 euro night, reading previous reviews decided jump, lucky enter hotel struck be

In [13]:
feed_df = pd.DataFrame({'Feedback':sample_df.Feedback,
                        'Sentiment':sentiment_ls})
feed_df

,Feedback,Sentiment
5065,stay stayed recently broadway manor inn oct. 2...,Positive
5245,great location friendly service ca n't tell co...,Positive
6408,simply wonderful wonderful 5 days beautiful ho...,Positive
7737,stay jianguo garden hotel beijing clean comfor...,Positive
1410,bad experience new york restaurant went ritz b...,Negative
...,...,...
4663,loved people resort wife vacationed secrets ex...,Positive
1450,"n't stay, second time staying argonaut celebra...",Positive
9010,place stay london lovely charme victorian hote...,Positive
1769,hotel gem location owners staff cleanliness am...,Positive


In [14]:
feed_df.to_csv("sample.csv")